In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
spotify_df = pd.read_csv('kaggleSPOTIFY.csv')

In [4]:
spotify_df.dropna(subset=['consolidates_genre_lists'], inplace=True)

In [5]:
spotify_df.isna().sum()

Unnamed: 0                  0
valence                     0
year                        0
acousticness                0
artists                     0
danceability                0
duration_ms                 0
energy                      0
explicit                    0
id                          0
instrumentalness            0
key                         0
liveness                    0
loudness                    0
mode                        0
name                        0
popularity                  0
release_date                0
speechiness                 0
tempo                       0
artists_upd_v1              0
artists_upd_v2              0
artists_upd                 0
artists_song                0
consolidates_genre_lists    0
dtype: int64

In [6]:
spotify_df.head()

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,['조정현'],[],['조정현'],조정현그아픔까지사랑한거야,['classic_korean_pop']
1,1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,['黑豹'],[],['黑豹'],黑豹DON'T BREAK MY HEART,"['chinese_indie', 'chinese_indie_rock']"
2,2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,['黃國隆'],[],['黃國隆'],黃國隆藝旦調,[]
3,3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"['黃國隆', '王秋玉']",[],"['黃國隆', '王秋玉']",黃國隆草螟弄雞公,[]
4,4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,['黃國隆'],[],['黃國隆'],黃國隆思想起,[]


In [7]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [8]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [9]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [10]:
mood_trans_np[10011]

array([ 3.37093e+05,  4.26000e-01,  1.01000e-01,  6.46000e-01,
        0.00000e+00,  3.58000e-01,  8.21000e-01, -1.10330e+01,
        1.32000e-01,  2.05703e+02])

In [11]:
df = pd.read_csv('data_moods.csv')

In [12]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [21]:

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

In [22]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [25]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [26]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Xveers\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Xveers\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\Users\Xveers\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "c:\Users\Xveers\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 928, in _fit
    self._fit_keras_model(
  File "c:\Users\Xveers\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 524, in _fi

In [23]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict()

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 4)

In [ ]:
y_preds

In [30]:
#Join the model and the scaler in a Pipeline
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#Fit the Pipeline
pip.fit(X2,encoded_y)

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 4)

In [31]:
def predict_mood(preds):
    
#     pipe = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#     #Fit the Pipeline
#     pipe.fit(X2,encoded_y)

    preds_features = np.array(preds[:]).reshape(-1,1).T

    #Predict the features of the song
    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")
    

In [32]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

KeyboardInterrupt: 

In [33]:
spotify_df.shape

(156410, 25)

In [34]:
print(len(res))

59


In [35]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [ ]:
spotify_df.to_csv('kaggleMusicMoodFinal.csv')

In [ ]:
res.count("Sad")

In [ ]:
res.count("Happy")

In [29]:
res.count("Energetic")

NameError: name 'res' is not defined

In [ ]:
res.count("Calm")

In [ ]:
spotify_df.shape